02.03.2020
###  Работа с данными 
## Семинар 1

Навигация:
 - [Задача про трафик](#Задача-1)
 - [Титаник](#Задача-2.-Титаник)
 - [Байесовский классификатор](#Задача-3.-Байесовский-классификатор-текстов)
      - [Реализация](#Нормальная-реализация-классификатора)

***

### Задача 1

2/3 трафика идет на прямые ссылки из поисковых систем, конверсия в просмотр 10%. Остальной трафик идет на ЦПБ, просмотров из него 30%. Найти общую конверию в просмотры.

Решение:

Событие $А$ - просмотр,
$H_1$ - прямой переход, $P(H_1) = 2/3$, $P(A|H_1) = 0.1$

$H_2$ - переход из ЦПБ, $P(H_2) = 1/3$, $P(A|H_2) = 0.3$

По формуле полной вероятности $P(A)= P(H_1)P(A|H_1)+P(H_2)P(A|H_2)$

In [5]:
pA = 2/3*0.1+1/3*0.3
print(f"Конверсия по обоим каналам равна {pA*100:.2f}%")

Конверсия по обоим каналам равна 16.67%


***

### Задача 2. Титаник

По данным колонок Class и Survived из датасета пассажиров Титаника найти вероятность выжить для пассажира 1 класса

$$
P(Survived = Yes | Class= 1st) = \frac{P(Class=1 | Survived = Yes) \cdot P(Survived = Yes)}{P(Class=1)}
$$

Решение:

В задачах с реальными данными вероятности заменяются частотами:

In [6]:
#загрузка датасета
import pandas as pd

df = pd.read_csv('Dataset.data', sep = ',', header=None, 
                 names=['Class','Age','Gender','Survived'], usecols = ['Class', 'Survived'])
df.head()

,Class,Survived
0,1st,yes
1,1st,yes
2,1st,yes
3,1st,yes
4,1st,yes


In [7]:
total_num = df.count()[0] #2201
pS = df[df.Survived == 'yes'].count()[0]/total_num #711

p1 = df[df.Class == '1st'].count()[0]/total_num #325

p1S = df[(df.Class == '1st') & (df.Survived=='yes')].count()[0]/df[df.Survived == 'yes'].count()[0] #203/711

pS1 = p1S*pS/p1
print(f"Вероятность выжить, если пассажир путешествовал первым классом {pS1*100:.2f} %")
print(f"Априорная вероятность выжить  {pS*100:.2f} %")

Вероятность выжить, если пассажир путешествовал первым классом 62.46 %
Априорная вероятность выжить  32.30 %


In [8]:
# а вообще можно было сделать сразу так

pS1a = df[(df.Class == '1st') & (df.Survived=='yes')].count()[0]/df[df.Class == '1st'].count()[0]
print(f"Вероятность выжить, если пассажир путешествовал первым классом {pS1a*100:.2f} %")

Вероятность выжить, если пассажир путешествовал первым классом 62.46 %


***

### Задача 3. Байесовский классификатор текстов

С помощью теоремы Байеса определить по тексту сообщения его класс (spam/ham).

**Обобщение предыдущей задачи на многомерный случай:**

$$
c_{MAP} = \arg \max_{\substack{c \in C}}P(c \mid X) = \arg \max_{\substack{c \in C}}\frac{P(c)P(X\mid c)}{P(X)}
$$

$$
c_{MAP} = \arg \max_{\substack{c \in C}}P(c)P(x_1\ldots x_n\mid c)
$$

$P(c)$ вычисляем как # сообщений класса/общее # сообщений.

$P(x_1\ldots x_n\mid c)$ в предположении о независимочти $x_i$ вычисляется как
$$
P(x_1\ldots x_n\mid c) = P(x_1 \mid c)\cdot \ldots \cdot P(x_n\mid c) = \prod_{i=1}^nP(x_i\mid c)
$$

Обучение наивного байесовского классификатора сводится к вычислению по корпусу текстов (тренировочных данных) относительных частот по категориям, тогда мы получаем т.н *multinomial bayes model*:
$$
\forall i,j: P(x_i \mid c_j) = \frac{n_{c_j}(x_i)}{\sum_{k\in V}n_{c_k}(x_i)}
$$
где $n_{c_j}(x_i)$ - количество раз, которое слово $x_i$ встречается в теме $c_j$, а $V$ - *словарь* корпуса документов, множество всех уникальных слов

**Тупой способ: без оформления в функцию**

In [4]:
pwd

'C:\\Users\\79850\\Documents\\MAI-data'

In [5]:
import pandas as pd

filename = '..\\SMSSpamCollection'
df = pd.read_csv(
    filename,
    sep='\t',
    header=None,
    names=['class','sms'])
df.head()

,class,sms
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
num_objects, num_features = df.shape

In [7]:
SPAM_CLASS = 'spam'
HAM_CLASS = 'ham'

In [8]:
spam_sms_num = (df['class'] == SPAM_CLASS).sum()
ham_sms_num = (df['class'] == HAM_CLASS).sum()


# априорные вероятности 
p_spam = spam_sms_num / num_objects
p_ham = ham_sms_num / num_objects

In [9]:
print(f'{p_spam:.4f}, {p_ham:.4f}')

0.1341, 0.8659


In [10]:
test_word = 'Free'.lower()

In [11]:
# предварительная обработка строки
import string

sms_example = df['sms'].values[5] # одна строка для примера
sms_example = ''.join([char for char in sms_example if char not in string.punctuation]) # удаляем знаки препинания
sms_example = ' '.join([word.lower() for word in sms_example.split()]) # приводим слова к нижнему регистру
sms_example

'freemsg hey there darling its been 3 weeks now and no word back id like some fun you up for it still tb ok xxx std chgs to send £150 to rcv'

In [12]:
# оформим это как функцию

def text_preprocess(sms_text: str):
    """Преобразоавние текста для анализа"""
    text_no_punctuation = ''.join([char for char in sms_text if char not in string.punctuation])
    text_lowercase = ' '.join([word.lower() for word in text_no_punctuation.split()])
     
    return text_lowercase

In [13]:
df = df.assign(processed_text = df['sms'].apply(text_preprocess)) # добавляем колонку с обработанным текстом
df.head()

,class,sms,processed_text
0,ham,"Go until jurong point, crazy.. Available only ...",go until jurong point crazy available only in ...
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...
3,ham,U dun say so early hor... U c already then say...,u dun say so early hor u c already then say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah i dont think he goes to usf he lives aroun...


In [14]:
#вероятность встретить тестовое слово в спаме

spam_test_word_entries = df[df['class']==SPAM_CLASS]['processed_text'].apply(lambda row: test_word in row).sum()
# вероятность встретить слово в не-спам смс
ham_test_word_entries = df[df['class'] == HAM_CLASS]['processed_text'].apply(lambda row: test_word in row).sum()

In [15]:
print(f'P(word="{test_word}"|class=spam)={spam_test_word_entries/spam_sms_num:.4f}')
print(f'P(word="{test_word}"|class=not_spam)={ham_test_word_entries/ham_sms_num:.4f}')

P(word="free"|class=spam)=0.2664
P(word="free"|class=not_spam)=0.0137


**Вывод:** слово "free"  - хороший маркер спама.

Обучение наивного байесовского классификатора сводится к вычислению по корпусу текстов (тренировочных данных) относительных частот по категориям, тогда мы получаем т.н *multinomial bayes model*:
$$
\forall i,j: P(x_i \mid c_j) = \frac{n_{c_j}(x_i)}{\sum_{k\in V}n_{c_k}(x_i)}
$$
где $n_{c_j}(x_i)$ - количество раз, которое слово $x_i$ встречается в теме $c_j$, а $V$ - *словарь* корпуса документов, множество всех уникальных слов


$$
c_{MAP} = \arg \max_{\substack{c \in C}}P(c)P(x_1\ldots x_n\mid c)
$$

$P(c)$ вычисляем как # сообщений класса/общее # сообщений.

$P(x_1\ldots x_n\mid c)$ в предположении о независимочти $x_i$ вычисляется как
$$
P(x_1\ldots x_n\mid c) = P(x_1 \mid c)\cdot \ldots \cdot P(x_n\mid c) = \prod_{i=1}^nP(x_i\mid c)
$$

### Нормальная реализация классификатора

In [32]:
class NaiveBayes:
    """Атрибуты"""
    #TODO - какие нужны атрибуты
    
    SPAM_CLASS
    HAM_CLASS
    
    #spam_num
    #ham_num
    
    
    def __init__(self, spam='spam', ham='ham'):
        # TODO атрибуты по умолчанию
        self.SPAM_CLASS = spam
        self.HAM_CLASS = ham         

    
    def sms_preprocess(sms_text: str):
        """Преобразоавние текста смс для анализа
        
        :param sms_text: текст отдельного смс
        """
        text_no_punctuation = ''.join([char for char in sms_text if char not in string.punctuation])
        word_list_lowecase = [word.lower() for word in text_no_punctuation.split()]
     
        return word_list_lowercase
    
    def words_num(processed_data: list, target: list, word: str, sms_class: str): 
        """ Подсчет появлений слова в конкретном классе """
        #TODO частоты???
        
        counter = 0
        masked_data = processed_data*(target==sms_class)
        for sms in masked_data:
            counter += sms.count(word)
        return counter
    

    def fit(self, data: list, target: list):
        """
        Обучение модели
        
        :param data: массив документов, каждый документ - объект типа str
        :param target: массив меток объектов
        :return: матрица Nx2 вероятностей P(x_i|c_j)
        """
        #TODO частоты в словаре?
        #TODO массив для условных вероятностей?
        #TODO в каком порядке будут идти слова в матрице? Сортировка? 
        processed_data = [sms_preprocess(sms) for sms in data]
        flat_data = [item for sublist in processed_data for item in sublist]
        vocab_sorted = list(set(flat_data)).sorted()
        
        self.spam_num=(target==SPAM_CLASS).sum()
        self.ham_num=(target==HAM_CLASS).sum()
        
        ham_freq = self.ham_num/len(data)
        spam_freq = self.spam_num/len(data)
        
        self.vocabulary = {item:flat_data.count(item) for item in set(flat_data)}
        
        probability_matrix = [[words_num(processed_data, target, word, self.SPAM_CLASS)/spam_num,
                               words_num(processed_data, target, word, self.HAM_CLASS)/ham_num] for word in vocab_sorted]
        
        
        print('Fitted!')
        return probability_matrix


    def predict(self, data: list):
        """
        :param data: массив документов, для каждого из которых нужно предсказать метку
        :return: массив предсказанных меток
        """    
        #TODO - поиск в словаре для предикта
        predicted = []
        processed_data = [text_preprocess(sms).split() for sms in data]
        for sms in processed_data:
            p_ham = 0
            p_spam = 0
            for word in sms:
                temp_p_ham =0
                temp_p_spam = 0
                
                
                for entry in fitted:
                    temp_p_ham+=entry[0]
                    temp_p_spam+=entry[1]
                
                
                p_ham+=temp_p_ham
                p_spam+=temp_p_spam
                
            p_ham = p_ham*self.ham_freq
            p_spam = p_spam*self.spam_freq
            
            if p_spam>p_ham:
                predicted.append(SPAM_CLASS)
            else:
                predicted.append(HAM_CLASS)
        
        return predicted

In [26]:
# прогон по нашему датасету

classifier = NaiveBayes()

In [27]:
%%time
fitted = classifier.fit(df['sms'].values[:300],df['class'].values[:300])

All fitted!
Wall time: 6.95 s


In [28]:
%%time
predicted = classifier.predict(df['sms'].values[300:500])

Wall time: 5.8 s


In [20]:
predicted


['ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',
 'ham',


### Проверка

In [102]:
target = df['class'].values[5000:]

In [103]:
comparison = [True if predicted[i]==target[i] else False for i in range(len(predicted)-1)]

In [104]:
len(predicted)

572

In [108]:
comparison.count(True)/len(comparison) # лол, 87%

0.8704028021015762

In [109]:
spam_sms_num = (df['class'] == SPAM_CLASS).sum()
ham_sms_num = (df['class'] == HAM_CLASS).sum()


In [110]:
ham_sms_num/(spam_sms_num + ham_sms_num)

0.8659368269921034

In [29]:
classifier.vocabulary

{'hopes': 1,
 'with': 25,
 'dont': 15,
 'available': 2,
 'repair': 1,
 'us': 4,
 'friends': 6,
 'g': 1,
 'i‘m': 2,
 'bus': 2,
 'offered': 1,
 '145': 1,
 '16': 3,
 'almost': 1,
 'side': 1,
 'club': 2,
 'know': 13,
 'haf': 2,
 'starwars3': 1,
 '2wks': 1,
 'ubandu': 1,
 'suggest': 1,
 'crave': 1,
 'child': 1,
 'wake': 1,
 'kallis': 1,
 'down': 6,
 '447801259231': 1,
 'kanoil': 1,
 'buying': 1,
 'sign': 1,
 'exam': 3,
 'okie': 1,
 'liked': 1,
 'usual': 1,
 'last': 3,
 'wanted': 1,
 'pleassssssseeeeee': 1,
 'partner': 1,
 'chores': 1,
 'get': 16,
 '9am11pm': 1,
 'however': 1,
 'wwwareyouuniquecouk': 1,
 'tncs': 1,
 'watching': 2,
 'important': 1,
 'housework': 1,
 'lucky': 1,
 'already': 8,
 'inviting': 1,
 'requests': 1,
 'no': 21,
 'worried': 1,
 'hours': 1,
 'fa': 2,
 'youll': 1,
 'theory': 1,
 'well': 6,
 'we': 13,
 'mmmmmm': 1,
 'eg': 2,
 'bday': 2,
 'crazy': 1,
 'dogg': 1,
 'code': 5,
 'incorrect': 1,
 'needed': 1,
 'pray': 1,
 'dats': 1,
 'module': 1,
 'later': 6,
 'q': 1,
 'four': 1